In [18]:
import sapien as sapien
from sapien.utils import Viewer
import numpy as np
from transforms3d.euler import euler2quat

from pathlib import Path
from shutil import rmtree
from PIL import Image

In [19]:

def create_car(
        scene: sapien.Scene,
        body_size=(1.0, 0.5, 0.25),
        tire_radius=0.15,
        joint_friction=0.0,
        joint_damping=0.0,
        density=1.0,
) -> sapien.physx.PhysxArticulation:
    body_half_size = np.array(body_size) / 2
    shaft_half_size = np.array([tire_radius * 0.1, tire_radius * 0.1, body_size[2] * 0.1])
    rack_half_size = np.array([tire_radius * 0.1, body_half_size[1] * 2.0, tire_radius * 0.1])
    builder: sapien.ArticulationBuilder = scene.create_articulation_builder()

    # car body (root of the articulation)
    body: sapien.LinkBuilder = builder.create_link_builder()  # LinkBuilder is similar to ActorBuilder
    body.set_name('body')
    body.add_box_collision(half_size=body_half_size, density=density)
    body.add_box_visual(half_size=body_half_size, material=[0.8, 0.6, 0.4])

    # front steering shaft
    front_shaft = builder.create_link_builder(body)
    front_shaft.set_name('front_shaft')
    front_shaft.set_joint_name('front_shaft_joint')
    front_shaft.add_box_collision(half_size=shaft_half_size, density=density)
    front_shaft.add_box_visual(half_size=shaft_half_size, material=[0.6, 0.4, 0.8])
    # The x-axis of the joint frame is the rotation axis of a revolute joint.
    front_shaft.set_joint_properties(
        'revolute',
        limits=[[-np.deg2rad(15), np.deg2rad(15)]],  # joint limits (for each DoF)
        # pose_in_parent refers to the relative transformation from the parent frame to the joint frame
        pose_in_parent=sapien.Pose(
            p=[(body_half_size[0] - tire_radius), 0, -body_half_size[2]],
            q=euler2quat(0, -np.deg2rad(90), 0)
        ),
        # pose_in_child refers to the relative transformation from the child frame to the joint frame
        pose_in_child=sapien.Pose(
            p=[0.0, 0.0, shaft_half_size[2]],
            q=euler2quat(0, -np.deg2rad(90), 0)
        ),
        friction=joint_friction,
        damping=joint_damping,
    )

    # back steering shaft (not drivable)
    back_shaft = builder.create_link_builder(body)
    back_shaft.set_name('back_shaft')
    back_shaft.set_joint_name('back_shaft_joint')
    back_shaft.add_box_collision(half_size=shaft_half_size, density=density)
    back_shaft.add_box_visual(half_size=shaft_half_size, material=[0.6, 0.4, 0.8])
    back_shaft.set_joint_properties(
        'fixed',
        limits=[],
        pose_in_parent=sapien.Pose(
            p=[-(body_half_size[0] - tire_radius), 0, -body_half_size[2]],
            q=euler2quat(0, -np.deg2rad(90), 0)
        ),
        pose_in_child=sapien.Pose(
            p=[0.0, 0.0, shaft_half_size[2]],
            q=euler2quat(0, -np.deg2rad(90), 0)
        ),
        friction=joint_friction,
        damping=joint_damping,
    )

    # front wheels
    front_wheels = builder.create_link_builder(front_shaft)
    front_wheels.set_name('front_wheels')
    front_wheels.set_joint_name('front_gear')
    # rack
    front_wheels.add_box_collision(half_size=rack_half_size, density=density)
    front_wheels.add_box_visual(half_size=rack_half_size, material=[0.8, 0.4, 0.6])
    # left wheel
    front_wheels.add_sphere_collision(pose=sapien.Pose(p=[0.0, rack_half_size[1] + tire_radius, 0.0]),
                                      radius=tire_radius, density=density)
    front_wheels.add_sphere_visual(pose=sapien.Pose(p=[0.0, rack_half_size[1] + tire_radius, 0.0]),
                                   radius=tire_radius,
                                   material=[0.4, 0.6, 0.8])
    # right wheel
    front_wheels.add_sphere_collision(pose=sapien.Pose(p=[0.0, -(rack_half_size[1] + tire_radius), 0.0]),
                                      radius=tire_radius, density=density)
    front_wheels.add_sphere_visual(pose=sapien.Pose(p=[0.0, -(rack_half_size[1] + tire_radius), 0.0]),
                                   radius=tire_radius,
                                   material=[0.4, 0.6, 0.8])
    # gear
    front_wheels.set_joint_properties(
        'revolute',
        limits=[[-np.inf, np.inf]],
        pose_in_parent=sapien.Pose(
            p=[0.0, 0, -(shaft_half_size[2] + rack_half_size[2])],
            q=euler2quat(0, 0, np.deg2rad(90))
        ),
        pose_in_child=sapien.Pose(
            p=[0.0, 0.0, 0.0],
            q=euler2quat(0, 0, np.deg2rad(90))
        ),
        friction=joint_friction,
        damping=joint_damping,
    )

    # back wheels
    back_wheels = builder.create_link_builder(back_shaft)
    back_wheels.set_name('back_wheels')
    back_wheels.set_joint_name('back_gear')
    # rack
    back_wheels.add_box_collision(half_size=rack_half_size, density=density)
    back_wheels.add_box_visual(half_size=rack_half_size, material=[0.8, 0.4, 0.6])
    # left wheel
    back_wheels.add_sphere_collision(pose=sapien.Pose(p=[0.0, rack_half_size[1] + tire_radius, 0.0]),
                                     radius=tire_radius, density=density)
    back_wheels.add_sphere_visual(pose=sapien.Pose(p=[0.0, rack_half_size[1] + tire_radius, 0.0]),
                                  radius=tire_radius,
                                  material=[0.4, 0.6, 0.8])
    # right wheel
    back_wheels.add_sphere_collision(pose=sapien.Pose(p=[0.0, -(rack_half_size[1] + tire_radius), 0.0]),
                                 radius=tire_radius, density=density)
    back_wheels.add_sphere_visual(pose=sapien.Pose(p=[0.0, -(rack_half_size[1] + tire_radius), 0.0]),
                                  radius=tire_radius,
                                  material=[0.4, 0.6, 0.8])
    # gear
    back_wheels.set_joint_properties(
        'revolute',
        limits=[[-np.inf, np.inf]],
        pose_in_parent=sapien.Pose(
            p=[0.0, 0, -(shaft_half_size[2] + rack_half_size[2])],
            q=euler2quat(0, 0, np.deg2rad(90))
        ),
        pose_in_child=sapien.Pose(
            p=[0.0, 0.0, 0.0],
            q=euler2quat(0, 0, np.deg2rad(90))
        ),
        friction=joint_friction,
        damping=joint_damping,
    )

    car = builder.build()
    car.set_name('car')
    return car

In [20]:
def get_joints_dict(articulation: sapien.physx.PhysxArticulation):
    joints = articulation.get_joints()
    joint_names =  [joint.name for joint in joints]
    assert len(joint_names) == len(set(joint_names)), 'Joint names are assumed to be unique.'
    return {joint.name: joint for joint in joints}

In [21]:
static_friction = 10.0
dynamic_friction = 10.0
restitution = 0.1
joint_friction = 0.0
joint_damping = 0.0

In [24]:
scene = sapien.Scene()
sapien.physx.set_default_material(static_friction, dynamic_friction, restitution)

scene.set_ambient_light([0.5, 0.5, 0.5])
scene.add_directional_light([0, 1, -1], [0.5, 0.5, 0.5], shadow=True)

timestep = 1 / 100.0
scene.set_timestep(timestep)

# ---------------------------------------------------------------------------- #
# Build a toy car
# ---------------------------------------------------------------------------- #
scene.add_ground(altitude=0)
car = create_car(scene, joint_friction=joint_friction, joint_damping=joint_damping)
car.set_pose(sapien.Pose(p=[0., 0., 0.34]))
print('The dof of the articulation is', car.dof)


# ---------------------------------------------------------------------------- #
# Camera
# Compute the camera pose by specifying forward(x), left(y) and up(z)
# ---------------------------------------------------------------------------- #

camera_position = np.array([-12, 0, 15])

camera_forward = -camera_position / np.linalg.norm(camera_position)
camera_left = np.cross([0, 0, 1], camera_forward)
camera_left = camera_left / np.linalg.norm(camera_left)
camera_up = np.cross(camera_forward, camera_left)

camera_matrix = np.eye(4)
camera_matrix[:3, :3] = np.stack([camera_forward, camera_left, camera_up], axis=1)
camera_matrix[:3, 3] = camera_position

near, far = 0.05, 100
width, height = 640, 480

camera = scene.add_camera(
    name="camera",
    width=width,
    height=height,
    fovy=1,
    near=near,
    far=far,
)
camera.entity.set_pose(sapien.Pose(camera_matrix))


# ---------------------------------------------------------------------------- #
# Control a toy car
# ---------------------------------------------------------------------------- #
joints = get_joints_dict(car)
print(joints.keys())
joints['front_shaft_joint'].set_drive_property(stiffness=1000.0, damping=0.0)  # front shaft
joints['front_gear'].set_drive_property(0.0, 1000.0)  # front gear
joints['back_gear'].set_drive_property(0.0, 0.0)  # back gear
limits = np.rad2deg(joints['front_shaft_joint'].get_limit()[0])


image_dir = Path("./demo_output")
if image_dir.exists():
    rmtree(image_dir)
image_dir.mkdir()


capture_fps = 25
capture_interval = int((1/capture_fps) / timestep)
capture_i = 0

position = 0.0  # position target of joints
velocity = 0.0  # velocity target of joints

accelerate = True
brake = False
left_turn = False
right_turn = False
reset = False

for steps in range(1000):
    
    if accelerate:
        velocity += 5.0
        print('velocity increases:', velocity)
        joints['front_gear'].set_drive_velocity_target(velocity)
        accelerate = False

    elif brake:
        velocity -= 5.0
        print('velocity decreases:', velocity)
        joints['front_gear'].set_drive_velocity_target(velocity)
        brake = False
        
    elif left_turn:
        position += 2
        position = np.clip(position, *limits)
        joints['front_shaft_joint'].set_drive_target(np.deg2rad(position))
        print('position increases:', position)
        left_turn = False
        
    elif right_turn:
        position -= 2
        position = np.clip(position, *limits)
        print('position decreases:', position)
        joints['front_shaft_joint'].set_drive_target(np.deg2rad(position))
        right_turn = False
        
    elif reset:
        position = 0
        velocity = 0.0
        print('reset')
        joints['front_shaft_joint'].set_drive_target(position)
        joints['front_gear'].set_drive_velocity_target(velocity)
        reset = False

    car.set_qf(car.compute_passive_force(True, True))
    scene.step()
    scene.update_render()

    if steps % 500 == 0:
        print('step:', steps)
        print('Pose', car.get_pose())
        print('Joint positions', car.get_qpos())
        print('Joint velocities', car.get_qvel())

    if steps % capture_interval == 0:
        camera.take_picture()
        rgba = camera.get_picture("Color")
        rgba_img = (rgba * 255).clip(0, 255).astype("uint8")
        rgba_pil = Image.fromarray(rgba_img)
        rgba_pil.save(f"{image_dir}/cam{capture_i:05}.png")
        capture_i += 1

print("Done.")

The dof of the articulation is 3
dict_keys(['', 'front_shaft_joint', 'back_shaft_joint', 'front_gear', 'back_gear'])
velocity increases: 5.0
step: 0
Pose Pose([0.000804568, 3.40993e-06, 0.34], [1, -7.66792e-07, -2.41776e-07, 2.82684e-06])
Joint positions [-9.6900527e-05  3.9860856e-02  5.3756316e-03]
Joint velocities [-0.02213549  4.9978986   0.5747797 ]
step: 500
Pose Pose([3.72525, 0.000130034, 0.34], [1, 4.96921e-08, 3.45803e-08, 1.7758e-05])
Joint positions [-6.5407448e-09 -1.9161078e-01 -2.9759017e-01]
Joint velocities [6.2633853e-04 4.9956470e+00 4.9970493e+00]
Done.


In [25]:
!/home/ajcd2020/miniforge3/envs/utilities/bin/ffmpeg -y -framerate 25 -i demo_output/cam%05d.png -pix_fmt yuv420p demo.mp4

ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.3.0 (conda-forge gcc 13.3.0-1)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646965297/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646965297/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646965297/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646965297/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1735646965297/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopen